In [2]:
import os
os.chdir('/home/seafood/wkdir/kg_trainer')

set the paramter

In [3]:

import os
import yaml


def load_hparam_str(hp_str):
    path = os.path.join('config', 'temp-restore.yaml')
    with open(path, 'w') as f:
        f.write(hp_str)
    return HParam(path)


def load_hparam(filename):
    stream = open(filename, 'r')
    docs = yaml.load_all(stream, Loader=yaml.FullLoader)
    hparam_dict = dict()
    for doc in docs:
        for k, v in doc.items():
            hparam_dict[k] = v
    return hparam_dict


def merge_dict(user, default):
    if isinstance(user, dict) and isinstance(default, dict):
        for k, v in default.items():
            if k not in user:
                user[k] = v
            else:
                user[k] = merge_dict(user[k], v)
    return user


class Dotdict(dict):
    """
    a dictionary that supports dot notation 
    as well as dictionary access notation 
    usage: d = DotDict() or d = DotDict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct=None):
        dct = dict() if not dct else dct
        for key, value in dct.items():
            if hasattr(value, 'keys'):
                value = Dotdict(value)
            self[key] = value


class HParam(Dotdict):

    def __init__(self, file):
        super(Dotdict, self).__init__()
        hp_dict = load_hparam(file)
        hp_dotdict = Dotdict(hp_dict)
        for k, v in hp_dotdict.items():
            setattr(self, k, v)
            
    __getattr__ = Dotdict.__getitem__
    __setattr__ = Dotdict.__setitem__
    __delattr__ = Dotdict.__delitem__


hp = HParam('config/default.yaml')

In [4]:
import argparse
import importlib




def import_class(module_and_class_name: str) -> type:
    """Import class from a module, e.g. 'text_recognizer.models.MLP'."""
    module_name, class_name = module_and_class_name.rsplit(".", 1)
    module = importlib.import_module(module_name)
    class_ = getattr(module, class_name)
    return class_


def setup_data_and_model_from_args(hp: argparse.Namespace):
    data_class = import_class(f"{hp.data_class_module}.{hp.data_class}")

    data = data_class(hp)
    #model = model_class(model_path=args.litmodel.final_path)
    model = None

    return data, model

data, model = setup_data_and_model_from_args(hp)

/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded succeed


[knowledge graph] based on Transformer


In [4]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('textattack/roberta-base-MNLI')

# define the entities and relations

entities =['tank','river']
relationships = ['forward', 'number']

# define the generator

def generate_kg(sentense):
    # fenci
    inputs = tokenizer(sentense, return_tensors='pt')
    # emotional
    outputs = model(**inputs)
    logits = outputs.logits.detach().numpy()
    sentiment = 'positive' if logits[0][0] >logits[0][1] else 'negative'

    # generate the kg according to the emotional

    if sentiment == 'positive':
        kg=[]
        for entity  in entities:
            for relation in relationships:
                if entity in sentense and relation in sentense:
                    kg.append((entity,relation,sentense.split(relation)[1].split()[0]))
        return kg
    else:
        return []

In [5]:
import os
from torch import optim
import pytorch_lightning as pl
from torchmetrics import Perplexity
from transformers import GPT2LMHeadModel
LR = 1e-3

class GPT2Chinese(pl.LightningModule):
    def __init__(self,args):
        super().__init__()

        self.gpt2 = GPT2LMHeadModel.from_pretrained(args.final_path)

        self.train_acc = Perplexity(ignore_index=-100)
        self.val_acc = Perplexity(ignore_index=-100)
        self.test_acc = Perplexity(ignore_index=-100)

    def forward(self,input_ids, length):
        out = self.gpt2(input_ids)
        return out


    def training_step(self, batch, batch_idx):
        x,y,logits, loss = self._run_on_batch(batch)
        #train step define the train loop

        self.train_acc(logits,y)
        self.log("train/loss", loss)
        self.log("train/acc", self.train_acc, on_step=False, on_epoch=True)

        return loss

    def _run_on_batch(self, batch, with_preds=False):
        x,y = batch
        out = self.gpt2(x,labels = y)
        loss, logits = out[:2]
        return x, y, logits, loss

    def validation_step(self,batch,batch_idx):
        x,y,logits, loss = self._run_on_batch(batch)

        self.val_acc(logits, y)
        self.log("validation/loss", loss, prog_bar=True, sync_dist=True)
        self.log("validation/acc", self.val_acc, on_step=False, on_epoch=True, prog_bar=True)


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(),lr=LR)
        return optimizer



In [6]:
lit_model_class = GPT2Chinese

lit_model = lit_model_class(args=hp.litmodel)

 

callbacks

In [7]:
from pathlib import Path
from pytorch_lightning.utilities.rank_zero import rank_zero_info, rank_zero_only

@rank_zero_only
def _ensure_logging_dir(experiment_dir):
    """Create the logging directory via the rank-zero process, if necessary."""
    Path(experiment_dir).mkdir(parents=True, exist_ok=True)

log_dir = Path("training") / "logs"
_ensure_logging_dir(log_dir)
logger = pl.loggers.TensorBoardLogger(log_dir)
experiment_dir = logger.log_dir

In [8]:
goldstar_metric = "validation/cer" if hp.litmodel.loss in ("transformer",) else "validation/loss"
filename_format = "epoch={epoch:04d}-validation.loss={validation/loss:.3f}"

checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=5,
        filename=filename_format,
        monitor=goldstar_metric,
        mode="min",
        auto_insert_metric_name=False,
        dirpath=experiment_dir,
        every_n_epochs=hp.chkpt.every_n_epochs,
    )

summary_callback = pl.callbacks.ModelSummary(max_depth=2)

In [9]:
trainer = pl.Trainer(devices=[0,],accelerator='cuda',
                     logger = logger,
                     callbacks=[summary_callback, checkpoint_callback])

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(lit_model, datamodule=data)

/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
You are using a CUDA device ('AMD Radeon RX 6800M') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 6/6 [00:00<00:00, 30.08it/s]


finish


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name             | Type            | Params
-----------------------------------------------------
0 | gpt2             | GPT2LMHeadModel | 102 M 
1 | gpt2.transformer | GPT2Model       | 102 M 
2 | gpt2.lm_head     | Linear          | 16.2 M
3 | train_acc        | Perplexity      | 0     
4 | val_acc          | Perplexity      | 0     
5 | test_acc         | Perplexity      | 0     
-----------------------------------------------------
102 M     Trainable params
0         Non-trainable params
102 M     Total params
408.275   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

: 

: 